In [13]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [14]:
TRAIN_DIR = 'dataset/images/train'
TEST_DIR = 'dataset/images/test'

In [15]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [16]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [17]:
print(train)

                                        image     label
0            dataset/images/train\angry\0.jpg     angry
1            dataset/images/train\angry\1.jpg     angry
2           dataset/images/train\angry\10.jpg     angry
3        dataset/images/train\angry\10002.jpg     angry
4        dataset/images/train\angry\10016.jpg     angry
...                                       ...       ...
28816  dataset/images/train\surprise\9969.jpg  surprise
28817  dataset/images/train\surprise\9985.jpg  surprise
28818  dataset/images/train\surprise\9990.jpg  surprise
28819  dataset/images/train\surprise\9992.jpg  surprise
28820  dataset/images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [18]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

.ipynb_checkpoints completed
angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [19]:
print(test)
print(test['image'])

                                      image     label
0       dataset/images/test\angry\10052.jpg     angry
1       dataset/images/test\angry\10065.jpg     angry
2       dataset/images/test\angry\10079.jpg     angry
3       dataset/images/test\angry\10095.jpg     angry
4       dataset/images/test\angry\10121.jpg     angry
...                                     ...       ...
7061  dataset/images/test\surprise\9806.jpg  surprise
7062  dataset/images/test\surprise\9830.jpg  surprise
7063  dataset/images/test\surprise\9853.jpg  surprise
7064  dataset/images/test\surprise\9878.jpg  surprise
7065   dataset/images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         dataset/images/test\angry\10052.jpg
1         dataset/images/test\angry\10065.jpg
2         dataset/images/test\angry\10079.jpg
3         dataset/images/test\angry\10095.jpg
4         dataset/images/test\angry\10121.jpg
                        ...                  
7061    dataset/images/test\surprise\9806.jpg
7062 

In [20]:
from tqdm.notebook import tqdm

In [21]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode = "grayscale" )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    

In [22]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [23]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [24]:
x_train = train_features/255.0
x_test = test_features/255.0

In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [27]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [28]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [29]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [30]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [31]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100


226/226 [==============================] - 129s 565ms/step - loss: 1.8226 - accuracy: 0.2431 - val_loss: 1.8051 - val_accuracy: 0.2587
Epoch 2/100
226/226 [==============================] - 130s 574ms/step - loss: 1.7766 - accuracy: 0.2613 - val_loss: 1.6901 - val_accuracy: 0.3133
Epoch 3/100
226/226 [==============================] - 129s 570ms/step - loss: 1.6821 - accuracy: 0.3180 - val_loss: 1.5850 - val_accuracy: 0.3900
Epoch 4/100
226/226 [==============================] - 129s 569ms/step - loss: 1.5667 - accuracy: 0.3843 - val_loss: 1.4325 - val_accuracy: 0.4526
Epoch 5/100
226/226 [==============================] - 128s 566ms/step - loss: 1.4874 - accuracy: 0.4190 - val_loss: 1.3846 - val_accuracy: 0.4690
Epoch 6/100
226/226 [==============================] - 129s 573ms/step - loss: 1.4370 - accuracy: 0.4460 - val_loss: 1.3221 - val_accuracy: 0.4962
Epoch 7/100
226/226 [==============================] - 128s 567ms/step - loss: 1.3949 - accuracy: 0.4631 - val_loss:

KeyboardInterrupt: 

In [32]:
model_json = model.to_json()
with open("emotiondetector.json2",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector2.h5")

C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
